# Get Simulation Data

This example shows how to get simulation data from a series of shot based on the CRIO data

In [46]:
import nptdms
from matplotlib import pyplot as plt
from north_diagnostics import Probe
import numpy as np
import pandas as pd

In [47]:
class Shot:
    def __init__(self, path: str, number: int, time_interval: tuple[float, float]):
        """
        Initialize a shot object.

        Parameters
        ----------
        number : int
            The shot number.
        time_interval : tuple[float, float]
            Start and end times (in seconds).
        """
        self.number = number
        self.time_interval = time_interval
        self.path = path
        self.gas_pressure = None
        self.toroidal_field_current = None
        self.heating_power = None

        self.vertical_field_current = 0

        vertical_field_lookup = {397: 6, 398: 6, 399: 6, 400: 12, 401: 12, 402: 12, 403: 15, 404: 15, 405: 15, 406: 18, 407: 18, 408: 18, 409: 21, 410: 21, 411: 21, 412: 21, 413: 24, 414: 24, 415: 24}
        if self.number in vertical_field_lookup:
            self.vertical_field_current = vertical_field_lookup[self.number]

        self.load_data()

    def load_data(self):
        """
        Load data from a file and extract gas pressure and toroidal field current
        for this shot and time interval.

        Parameters
        ----------
        filepath : str
            Path to the CSV data file.
        """
        tdms_file = nptdms.TdmsFile.read(f"{self.path}/CRIO{self.number}.tdms")



        # Store attributes as averages over the time interval
        self.gas_pressure = self.mean_in_interval(tdms_file['Data']['Time'][:], tdms_file['Data']['Pressure'][:]*100) # Pa
        self.toroidal_field_current = self.mean_in_interval(tdms_file['Data']['Time'][:], tdms_file['Data']['I_TF'][:]) # A
        self.heating_power = self.mean_in_interval(tdms_file['Data']['Time'][:], tdms_file['Data']['LFSset'][:]*450/3000) # W
    
    def mean_in_interval(self, time, values):
        """
        Compute mean of values in given time interval with interpolation
        at the boundaries.
        """
        # Ensure arrays are numpy
        time = np.asarray(time)
        values = np.asarray(values)
        t_min, t_max = self.time_interval

        # Mask for points strictly inside interval
        mask = (time > t_min) & (time < t_max)

        # Values inside
        t_inside = time[mask]
        v_inside = values[mask]

        # Interpolate values at boundaries
        v_tmin = np.interp(t_min, time, values)
        v_tmax = np.interp(t_max, time, values)

        # Combine: boundary + inside
        t_all = np.concatenate(([t_min], t_inside, [t_max]))
        v_all = np.concatenate(([v_tmin], v_inside, [v_tmax]))

        # print(f"{v_all = }")

        # fig, ax = plt.subplots()
        # ax.plot(time, values, label='Original Data')
        # ax.fill_between(t_all, v_all, alpha=0.3, label='Integration Area')
        # ax.axvline(t_min, color='red', linestyle='--', label='t_min')
        # ax.axvline(t_max, color='green', linestyle='--', label='t_max')
        # ax.set_xlabel('Time (ms)')
        # ax.set_ylabel('Value')
        # ax.legend()

        # Compute mean by trapezoidal integration (time-weighted)
        mean_val = np.trapezoid(v_all, t_all) / (t_max - t_min)
        return mean_val

    def __repr__(self):
        return (f"<Shot {self.number} | "
                f"Interval={self.time_interval} ms, "
                f"Heating Power={self.heating_power:0.0f} W, "
                f"Gas Pressure={self.gas_pressure} Pa, "
                f"Vertical Field Current={self.vertical_field_current} A, "
                f"Toroidal Field Current={self.toroidal_field_current} A>")


In [48]:
path = "/mnt/c/Users/alec/Data/NORTH/ProbeSim_2025/Helium/ControlData"

# Power Scan

In [ ]:
shot_list = [347,348,349,357,359,361,363,364,366,367]
time_intervals = [(300, 310), (600, 610)]

power, pressure, tf_current, vf_current = [], [], [], []

for shot_number in shot_list:
    for interval in time_intervals:
        shot = Shot(path, shot_number, interval)
        power.append(shot.heating_power)
        pressure.append(shot.gas_pressure)
        tf_current.append(shot.toroidal_field_current)
        vf_current.append(shot.vertical_field_current)
        print(shot)

<Shot 347 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.008497836504581732 Pa, Vertical Field Current=0 A, Toroidal Field Current=1071.3836669921875 A>
<Shot 347 | Interval=(600, 610) ms, Heating Power=450 W, Gas Pressure=0.009343814035872967 Pa, Vertical Field Current=0 A, Toroidal Field Current=934.0291821289062 A>
<Shot 348 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.008257125519506251 Pa, Vertical Field Current=0 A, Toroidal Field Current=1067.973938598633 A>
<Shot 348 | Interval=(600, 610) ms, Heating Power=450 W, Gas Pressure=0.009088425547269487 Pa, Vertical Field Current=0 A, Toroidal Field Current=931.6329650878906 A>
<Shot 349 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.007691421963108081 Pa, Vertical Field Current=0 A, Toroidal Field Current=1066.0503387451172 A>
<Shot 349 | Interval=(600, 610) ms, Heating Power=450 W, Gas Pressure=0.00896459200910528 Pa, Vertical Field Current=0 A, Toroidal Field Current=928.9937133789062 

In [ ]:
print("P: {" + ", ".join(f"{x:.0f}" for x in power) + "}")
print("I_TF: {" + ", ".join(f"{x:.0f}" for x in tf_current) + "}")
print("I_vertical: {" + ", ".join(f"{x:.0f}" for x in vf_current) + "}")
print("p_neut: {" + ", ".join(f"{x:.2e}" for x in pressure) + "}")

P: {450, 450, 450, 450, 450, 450, 400, 400, 400, 400, 400, 400, 350, 350, 350, 350, 350, 350, 300, 300}
I_TF: {1020, 934, 1017, 932, 1015, 929, 999, 915, 999, 915, 1003, 917, 999, 914, 1018, 931, 1011, 926, 1009, 923}
I_vertical: {0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}
p_neut: {9.17e-03, 9.34e-03, 8.59e-03, 9.09e-03, 8.44e-03, 8.96e-03, 7.92e-03, 8.31e-03, 8.38e-03, 8.28e-03, 8.02e-03, 8.41e-03, 7.93e-03, 7.97e-03, 7.64e-03, 7.95e-03, 7.80e-03, 8.01e-03, 7.59e-03, 7.91e-03}


# B_vert Scan

In [65]:
shot_list = [394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 413, 414, 415, 416, 417, 418]
time_intervals = [(150, 160)]

power, pressure, tf_current, vf_current = [], [], [], []

for shot_number in shot_list:
    for interval in time_intervals:
        shot = Shot(path, shot_number, interval)
        power.append(shot.heating_power)
        pressure.append(shot.gas_pressure)
        tf_current.append(shot.toroidal_field_current)
        vf_current.append(shot.vertical_field_current)
        print(shot)

<Shot 394 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.016730726858553226 Pa, Vertical Field Current=0 A, Toroidal Field Current=1061.8341064453125 A>
<Shot 395 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.015718687127015095 Pa, Vertical Field Current=0 A, Toroidal Field Current=1060.2548657226562 A>
<Shot 396 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.016362854863377886 Pa, Vertical Field Current=0 A, Toroidal Field Current=1059.1991674804688 A>
<Shot 397 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.016885209516077714 Pa, Vertical Field Current=6 A, Toroidal Field Current=1058.156611328125 A>
<Shot 398 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.016868435758496628 Pa, Vertical Field Current=6 A, Toroidal Field Current=1056.4591503906252 A>
<Shot 399 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.016011801921893682 Pa, Vertical Field Current=6 A, Toroidal Field Current=1055.7647802734

In [66]:
print("P: {" + ", ".join(f"{x:.0f}" for x in power) + "}")
print("I_TF: {" + ", ".join(f"{x:.0f}" for x in tf_current) + "}")
print("I_vertical: {" + ", ".join(f"{x:.0f}" for x in vf_current) + "}")
print("p_neut: {" + ", ".join(f"{x:.2e}" for x in pressure) + "}")

P: {450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450}
I_TF: {1062, 1060, 1059, 1058, 1056, 1056, 1055, 1054, 1053, 1052, 1051, 1050, 1050, 1049, 1049, 1048, 1047, 1048, 1048, 1047, 1046, 1045, 1045, 1044}
I_vertical: {0, 0, 0, 6, 6, 6, 12, 12, 12, 15, 15, 15, 18, 18, 18, 21, 21, 21, 24, 24, 24, 0, 0, 0}
p_neut: {1.67e-02, 1.57e-02, 1.64e-02, 1.69e-02, 1.69e-02, 1.60e-02, 1.65e-02, 1.70e-02, 1.70e-02, 1.68e-02, 1.64e-02, 1.64e-02, 1.67e-02, 1.58e-02, 1.69e-02, 1.65e-02, 1.66e-02, 1.61e-02, 1.64e-02, 1.69e-02, 1.68e-02, 1.67e-02, 1.64e-02, 1.66e-02}


# Pressure Scan

In [60]:
shot_list = [347,348,349,350,351,352,353,354,355]
time_intervals = [(150, 160), (600, 610)]

power, pressure, tf_current, vf_current = [], [], [], []

for shot_number in shot_list:
    for interval in time_intervals:
        shot = Shot(path, shot_number, interval)
        power.append(shot.heating_power)
        pressure.append(shot.gas_pressure)
        tf_current.append(shot.toroidal_field_current)
        vf_current.append(shot.vertical_field_current)
        print(shot)

<Shot 347 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.008497836504581732 Pa, Vertical Field Current=0 A, Toroidal Field Current=1071.3836669921875 A>
<Shot 347 | Interval=(600, 610) ms, Heating Power=450 W, Gas Pressure=0.009343814035872967 Pa, Vertical Field Current=0 A, Toroidal Field Current=934.0291821289062 A>
<Shot 348 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.008257125519506251 Pa, Vertical Field Current=0 A, Toroidal Field Current=1067.973938598633 A>
<Shot 348 | Interval=(600, 610) ms, Heating Power=450 W, Gas Pressure=0.009088425547269487 Pa, Vertical Field Current=0 A, Toroidal Field Current=931.6329650878906 A>
<Shot 349 | Interval=(150, 160) ms, Heating Power=450 W, Gas Pressure=0.007691421963108081 Pa, Vertical Field Current=0 A, Toroidal Field Current=1066.0503387451172 A>
<Shot 349 | Interval=(600, 610) ms, Heating Power=450 W, Gas Pressure=0.00896459200910528 Pa, Vertical Field Current=0 A, Toroidal Field Current=928.9937133789062 

In [61]:
print("P: {" + ", ".join(f"{x:.0f}" for x in power) + "}")
print("I_TF: {" + ", ".join(f"{x:.0f}" for x in tf_current) + "}")
print("I_vertical: {" + ", ".join(f"{x:.0f}" for x in vf_current) + "}")
print("p_neut: {" + ", ".join(f"{x:.2e}" for x in pressure) + "}")

P: {450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450}
I_TF: {1071, 934, 1068, 932, 1066, 929, 1064, 927, 1062, 925, 1060, 924, 1057, 921, 1056, 920, 1054, 919}
I_vertical: {0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}
p_neut: {8.50e-03, 9.34e-03, 8.26e-03, 9.09e-03, 7.69e-03, 8.96e-03, 1.61e-02, 1.68e-02, 1.57e-02, 1.72e-02, 1.64e-02, 1.71e-02, 2.43e-02, 2.56e-02, 2.41e-02, 2.62e-02, 2.37e-02, 2.52e-02}
